In [1]:
import requests
import re
from bs4 import BeautifulSoup 
import pandas as pd
import numpy as np

def baha_picture(url,page_number,gp_number):
    # load
    data = requests.get(url)
    soup = BeautifulSoup(data.text,'lxml')
    # Find page
    di = soup.find('div',id="BH-master")
    s = di.find_all(href = re.compile("page="))
    l = []
    for i in s:
        try:
            l.append(int(i.contents[0]))
        except ValueError:
            continue
    page = max(l)
    if page_number > page:
        page_number = page
    # Get all pages of url
    pageList = []
    if len(url.split('page')) == 1:
        url = url.split("php?")[0]+"php?"+"page=1&"+url.split("php?")[1]
    for i in range(1,page+1):
        pageList.append(url.split("page=")[0]+"page="+str(i)+"&bsn"+url.split("&bsn")[1])
        
    # Get gp / bp / floor / picture
    gp = []
    bp = []
    a_list = []
    pic_list = []
    floor_list = []
    n = 0
    for page in pageList[:page_number]:
        url = page
        data = requests.get(url)
        soup = BeautifulSoup(data.text,'lxml')
        floor = soup.find_all('a',class_ ="count tippy-gpbp-list")
        # gp
        for i in floor[::2]:
            if i.contents[0] == '爆':
                gp.append(1000)
            else:
                gp.append(int(i.contents[0])) 
        # bp
        for i in floor[1::2]:
            if i.contents[0] == 'X':
                bp.append(1000)
            elif i.contents[0] == '-':
                bp.append(0)
            else:
                bp.append(int(i.contents[0]))
        #floor
        alive = soup.find_all('a',class_ = "floor")
        for i in alive:
            a_list.append(i.contents[0])
        # picture
        picture = soup.find_all('div',class_ = "c-section__main c-post ")
        for i in range(len(picture)):
            pic_list.append([])
            try:
                #pic_list[i].append(picture[i].find('a',class_="floor").contents[0])
                [pic_list[n+i].append(j['href']) for j in picture[i].find_all('a',href = re.compile("\/truth.bahamut.com.tw\/"))]
            except IndexError:
                continue
        n+=len(picture)
    # Prepare for Dataframe
    a_list = np.array(a_list).reshape(-1,1)
    bp = np.array(bp).reshape(-1,1)
    gp = np.array(gp).reshape(-1,1)
    diff = gp - bp
    floor_list = np.array(floor_list).reshape(-1,1)
    pic_list = np.array(pic_list).reshape(-1,1)

    # Turn into Dataframe
    dataf = np.concatenate((bp,diff),axis = 1)
    dataf = np.concatenate((gp,dataf),axis = 1)
    dataf = np.concatenate((a_list,dataf),axis = 1)
    dataf1 = np.concatenate((dataf,pic_list),axis = 1)
    Liketable = pd.DataFrame(dataf1,columns = ("樓層","推","噓","差","網址"))
    Liketable_simple = pd.DataFrame(dataf,columns = ("樓層","推","噓","差"))
    pd.set_option('max_colwidth', 200)

    # Turn into int
    Liketable["推"] = pd.to_numeric(Liketable["推"])
    Liketable["噓"] = pd.to_numeric(Liketable["噓"])
    Liketable["差"] = pd.to_numeric(Liketable["差"])
    Liketable_simple["推"] = pd.to_numeric(Liketable_simple["推"])
    Liketable_simple["噓"] = pd.to_numeric(Liketable_simple["噓"])
    Liketable_simple["差"] = pd.to_numeric(Liketable_simple["差"])
    
    # Filter
    filter1 = (Liketable_simple['差'] >= gp_number)
    print(Liketable_simple[filter1])

    filter1 = (Liketable['差'] >= gp_number)
    for i in Liketable[filter1]["樓層"]:
        fliter = (Liketable['樓層']== i)
        print(i)
        for x in Liketable[fliter]["網址"]:
            if x == []:
                print("\t沒圖啦")
            else:
                [print(z) for z in x]
# Set up url 
url = "https://forum.gamer.com.tw/C.php?page=1&bsn=60076&snA=3144471&locked=F&gothis=54827297#54827297"

# Get 1-10 pages of the url of pictures (Gp - Bp) > 50
baha_picture(url,10,50)

        樓層    推    噓    差
0       樓主  739   10  729
2      6 樓  242    9  233
12    16 樓  441    0  441
18    23 樓  593    0  593
26    31 樓  343   37  306
29    36 樓  475    0  475
38    47 樓   53    0   53
40    49 樓  125    0  125
47    56 樓   95    0   95
49    59 樓  195    0  195
51    61 樓   55    0   55
52    62 樓   68    0   68
53    64 樓   86    0   86
55    66 樓   59    0   59
57    68 樓   50    0   50
66    78 樓  100    0  100
71    83 樓  129    0  129
74    86 樓   61    0   61
79    92 樓  179  123   56
109  126 樓   56    0   56
136  155 樓   65    0   65
137  157 樓   68    0   68
樓主
https://truth.bahamut.com.tw/s01/201601/f24e834da66aa649fb2b236c7b2377d1.JPG
6 樓
https://truth.bahamut.com.tw/s01/201402/0d5c2e423861fb9c9de1fec8a054c3a2.JPG
16 樓
https://truth.bahamut.com.tw/s01/201601/e48d68bfc3a16048fd4b27594403bb76.JPG
23 樓
https://truth.bahamut.com.tw/s01/201601/77a711f23653b34abda15e0091988764.JPG
31 樓
https://truth.bahamut.com.tw/s01/201601/9c0c06f8354b6d251073315048cd3a7f